In [11]:
import re

from nltk.corpus import wordnet as wn

import unidecode

from time import sleep

In [12]:
# Creer liste mot-clé

list_mot = ['converse', 'rubrique']

list_syn_rub = { re.sub('[^a-zA-Z0-9 \n\.]', ' ', unidecode.unidecode(lem)).replace(" ","") for syn in wn.synsets(list_mot[1], lang='fra') for lem in syn.lemma_names('fra') }

list_syn = {
    'ask': {'appelle', 'suis', 'nomme', 'est'},
    'rub': list_syn_rub
}
list_syn

{'ask': {'appelle', 'est', 'nomme', 'suis'},
 'rub': {'chapitre',
  'colonne',
  'entete',
  'glose',
  'intitule',
  'ligne',
  'partie',
  'pilier',
  'rubrique',
  'section',
  'tete',
  'titre'}}

In [13]:
rubriques = [
    "Je me suis préinscrit. Que reste-t-il à faire ?", 
    "Dés que je finis de m'inscrire en ligne , est ce que je peux aller recupérer ma carte d'étudiant?",
    "Où dois je aller pour récupérer ma carte?",
    "Si j'ai un souci lors de mon inscription en ligne, à qui je m'adresse? Vers qui dois je me tourner?"
]

In [14]:
# Building dictionary of Intents & Keywords

keywords={}

keywords_dict={}

# Defining keys in the keywords dictionary

keywords['ask']=[]
keywords['rub']=[]

# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 

for synonym in list(list_syn['ask']):

    keywords['ask'].append('.*\\b'+synonym+'\\b.*')


# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 

for synonym in list(list_syn['rub']):

    keywords['rub'].append('.*\\b'+synonym+'\\b.*')


for intent, keys in keywords.items():


    # Joining the values in the keywords dictionary with the OR (|) operator updating them in keywords_dict dictionary

    keywords_dict[intent]=re.compile('|'.join(keys))
    

print(keywords)
print (keywords_dict)





{'ask': ['.*\\best\\b.*', '.*\\bnomme\\b.*', '.*\\bappelle\\b.*', '.*\\bsuis\\b.*'], 'rub': ['.*\\bintitule\\b.*', '.*\\bsection\\b.*', '.*\\brubrique\\b.*', '.*\\bchapitre\\b.*', '.*\\bpartie\\b.*', '.*\\bentete\\b.*', '.*\\bpilier\\b.*', '.*\\bcolonne\\b.*', '.*\\btete\\b.*', '.*\\btitre\\b.*', '.*\\bligne\\b.*', '.*\\bglose\\b.*']}
{'ask': re.compile('.*\\best\\b.*|.*\\bnomme\\b.*|.*\\bappelle\\b.*|.*\\bsuis\\b.*'), 'rub': re.compile('.*\\bintitule\\b.*|.*\\bsection\\b.*|.*\\brubrique\\b.*|.*\\bchapitre\\b.*|.*\\bpartie\\b.*|.*\\bentete\\b.*|.*\\bpilier\\b.*|.*\\bcolonne\\b.*|.*\\btete\\b.*|.*\\btitre\\b.*|.*\\bligne\\b.*|.*\\bglo)}


In [15]:
# Building a dictionary of responses

rub_res = "\n".join([str(i+1)+". "+rubriques[i] for i in range(len(rubriques))])

responses={

    'ask':['Puis-je connaitre votre prénom, SVP ?', 'Votre nom, SVP ?', 'Et enfin votre email, SVP ?'],

    'rub':rub_res,

    'fallback':'Je ne comprends pas bien. Pourriez-vous répéter, SVP ?',

}
print(responses)

{'ask': ['Puis-je connaitre votre prénom, SVP', 'Puis-je connaitre votre nom, SVP', 'Votre email, SVP'], 'timings': "1. Je me suis préinscrit. Que reste-t-il à faire ?\n2. Dés que je finis de m'inscrire en ligne , est ce que je peux aller recupérer ma carte d'étudiant?\n3. Où dois je aller pour récupérer ma carte?\n4. Si j'ai un souci lors de mon inscription en ligne, à qui je m'adresse? Vers qui dois je me tourner?", 'fallback': 'Je ne comprends pas bien. Pourriez-vous répéter ?'}


In [27]:
rub_res = {
    1: ["Après validation de votre préinscription en ligne, vous allez recevoir par email une message vous informant de la disponibilité en ligne (dans l'Espace Etudiant de la préinscription en ligne) de votre fiche d'inscription et de votre certificat d'inscription.",
        "Vous pouvez, par la suite, au niveau du service de scolarité de votre établissement muni de votre dossier d'inscription complet pour retirer votre carte d'étudiant."],
    2: ["Aprés la préinscription en ligne, présentez vous à votre scolarité muni de votre dossier.", "Apres vérification, les agents de scolarité procéderont à la confection de votre carte d'étudiant et vous la remettrons."],
    3: ["A la scolarité de votre établissement (faculté, école,...)."],
    4: ["Un service support est à votre disposition pour tout probléme concernant la préinscription en ligne.", "L'équipe support vous recevra au DRTP (ex Centre de calcul), sise derrière la Faculté des Sciences et Techniques (Fac Sciences).",
        "Disponible en semaine de 09h à 13h et de 15h à 17h  et les samedis de 09h à 12h, vous pouvez également les joindre au 77 856 54 90 ou par mail : support@ucad.edu.sn"]
}

In [25]:
def searchText(inputText, step):
    inputText = inputText.lower()
    sizeText = len(inputText.split(" "))
    if(sizeText == 1):
        return (inputText, ++step)
    
    for intent,pattern in keywords_dict.items():

        # Using the regular expression search function to look for keywords in user input
        if re.search(pattern, inputText): 

            # if a keyword matches, select the corresponding intent from the keywords_dict dictionary
            matched_intent=intent 
    
    if matched_intent in responses:
        info = inputText.split(" ")[-1]
        return (info, ++step)
    
    return ("", step)

In [26]:
print('Bonjour')
sleep(2)
ask_step = 0
choice = 0
list_info = []
while(True):
    try:
        
        # Defining the Chatbot's exit condition
        if ask_step > 2 and choice == -1: 
            print ("Merci de votre visite.")
            break  

        if ask_step <= 2:
            print(responses['ask'][ask_step])
            res = input("")
            info, ask_step = searchText(res, ask_step)
            if info != "": list_info.append(info.capitalize) else: raise ValueError("")
            
        if ask_step > 2 and choice != -1:
            print(f'Pour vous aider {list_info[0]} {list_info[1]},\n Merci de choisir parmis les rubrique suivantes :')
            sleep(1)
            print(responses['rub'])
            choice = int(input())
            if(choice > 0 and choice < 5):
                for resp in rub_res[choice]:
                    print(resp)
                    sleep(1)
                break
            else: raise ValueError("")
            
            
            
    except Exception as e:
        print(responses['fallback'])
    break

Bonjour
Puis-je connaitre votre prénom, SVP
Mon nom est testa
ask
